In [ ]:
# Import necessary packages
import pandas as pd
import numpy as np
from omegaconf import OmegaConf

In [ ]:
# Global configs
yaml_cfg = """
train_data_path: ./raw_data/hm_demo.csv
seed: 42
predict_target: outcome # outcome/LOS
"""
config = OmegaConf.create(yaml_cfg)
print(config.seed)


In [ ]:
# Read raw data
df_demo: pd.DataFrame = pd.read_csv("./raw_data/hm_demo.csv", encoding='unicode_escape', sep=",")
df_labtest: pd.DataFrame = pd.read_csv("./raw_data/hm_labtest.csv", encoding='unicode_escape', sep=",")


In [ ]:
df_demo['SEX'].replace('MALE', 1, inplace=True)
df_demo['SEX'].replace('FEMALE', 0, inplace=True)
df_demo.rename(columns={'IDINGRESO': 'PATIENT_ID', 'EDAD': 'AGE', 'DIAGNOSTICO_ING': 'outcome'}, inplace=True)
df_demo['outcome'].replace('COVID19 - POSITIVO', 1, inplace=True)
df_demo['outcome'].replace('COVID19 - PENDIENTE', 0, inplace=True)

cols_str = """AGE	TA_MAX_PRIMERA_URG	TA_MIN_PRIMERA_URG	TEMP_PRIMERA_URG	FC_PRIMERA_URG	SAT_02_PRIMERA_URG	GLU_PRIMERA_URG	DIURESIS_PRIMERA_URG	HORA_CONSTANTES_ULTIMA_URG	TA_MAX_ULTIMA_URG	TA_MIN_ULTIMA_URG	TEMP_ULTIMA_URG	FC_ULTIMA_URG	SAT_02_ULTIMA_URG	GLU_ULTIMA_URG"""
cols = cols_str.strip().split()
df_demo[cols] = df_demo[cols].replace([0, 0.0, '0'], np.nan)

In [ ]:
df_demo.to_csv('train.csv', index=False)
df_demo